In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix)

import joblib


In [3]:
df = pd.read_csv("../Data/processed/dataset_clean.csv")
df.head()

,Temperature,Humidity,pH,Rainfall,Label,Category
0,20.879744,82.002744,6.502985,202.935536,Rice,Cereal
1,21.770462,80.319644,7.038096,226.655537,Rice,Cereal
2,23.004459,82.320763,7.840207,263.964248,Rice,Cereal
3,26.491096,80.158363,6.980401,242.864034,Rice,Cereal
4,20.130175,81.604873,7.628473,262.717340,Rice,Cereal


In [4]:
# Definir X (variables predictoras) y y (etiqueta a predecir)
# X contiene las características ambientales que explican el cultivo:
# Temperature, Humidity, pH y Rainfall
# y contiene la columna Label, que es el tipo de cultivo que queremos predecir

X = df[["Temperature", "Humidity", "pH", "Rainfall"]]
y = df["Label"]

X.head(), y.head()

(   Temperature   Humidity        pH    Rainfall
 0    20.879744  82.002744  6.502985  202.935536
 1    21.770462  80.319644  7.038096  226.655537
 2    23.004459  82.320763  7.840207  263.964248
 3    26.491096  80.158363  6.980401  242.864034
 4    20.130175  81.604873  7.628473  262.717340,
 0    Rice
 1    Rice
 2    Rice
 3    Rice
 4    Rice
 Name: Label, dtype: object)

In [5]:
# Dividir los datos en entrenamiento (80%) y prueba (20%)
# Se usa stratify=y para mantener la misma proporción de cultivos en ambos conjuntos.
# random_state=42 asegura que la división sea reproducible.

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [6]:
# Crear el escalador
scaler = StandardScaler()

# Ajustar con los datos de entrenamiento y transformar
X_train_scaled = scaler.fit_transform(X_train)

# Transformar los datos de prueba con la misma escala
X_test_scaled = scaler.transform(X_test)

In [7]:
# Modelo KNN: clasifica cada muestra según los k vecinos más cercanos.
# Requiere datos escalados porque usa distancias. Se usa como modelo
# basado en similitud para comparar contra otros algoritmos.

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

knn_pred = knn.predict(X_test_scaled)


print("    KNN    "  )
print("Accuracy:", accuracy_score(y_test, knn_pred))
print(classification_report(y_test, knn_pred))
# Random Forest: modelo basado en múltiples árboles de decisión votando en conjunto.
# n_estimators=200 mejora la estabilidad y precisión del modelo. No requiere escalado.

    KNN    
Accuracy: 0.8521428571428571
                  precision    recall  f1-score   support

        Aleovera       0.90      0.95      0.93        20
           Apple       0.86      0.95      0.90        20
        Arecanut       0.95      0.90      0.92        20
     Ashwagandha       1.00      1.00      1.00        20
           Bajra       1.00      1.00      1.00        20
          Banana       0.63      0.95      0.76        20
        Beetroot       0.91      1.00      0.95        20
       Blackgram       0.62      0.80      0.70        20
         Brinjal       0.79      0.75      0.77        20
        Broccoli       0.82      0.90      0.86        20
Button Mushrooms       0.50      0.45      0.47        20
         Cabbage       0.90      0.90      0.90        20
        Capsicum       0.87      1.00      0.93        20
          Carrot       0.95      1.00      0.98        20
      Cashewnuts       0.94      0.80      0.86        20
     Cauliflower       0.67   

In [ ]:
cm_knn = confusion_matrix(y_test, knn_pred)
plt.figure(figsize=(8,5))
sns.heatmap(cm_knn, annot=True, fmt=".0f")
plt.title("KNN - Matriz de Confusión")
plt.xlabel("Predicho")
plt.ylabel("Real")

In [8]:
# Random Forest: modelo basado en múltiples árboles de decisión votando en conjunto.
# n_estimators=200 mejora la estabilidad y precisión del modelo. No requiere escalado.

rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)

print("      Random Forest      ")
print("Accuracy:", accuracy_score(y_test, rf_pred))
print(classification_report(y_test, rf_pred))

      Random Forest      
Accuracy: 0.9564285714285714
                  precision    recall  f1-score   support

        Aleovera       1.00      1.00      1.00        20
           Apple       1.00      0.95      0.97        20
        Arecanut       1.00      0.90      0.95        20
     Ashwagandha       1.00      1.00      1.00        20
           Bajra       1.00      1.00      1.00        20
          Banana       1.00      1.00      1.00        20
        Beetroot       1.00      1.00      1.00        20
       Blackgram       0.83      0.95      0.88        20
         Brinjal       1.00      1.00      1.00        20
        Broccoli       1.00      1.00      1.00        20
Button Mushrooms       0.64      0.45      0.53        20
         Cabbage       1.00      1.00      1.00        20
        Capsicum       0.90      0.90      0.90        20
          Carrot       1.00      1.00      1.00        20
      Cashewnuts       0.89      0.85      0.87        20
     Cauliflower

In [ ]:
rf_base = RandomForestClassifier(random_state=42)

# 2. Malla de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# 3. GridSearch usando LOS DATOS ESCALADOS
grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Entrenamiento
grid_search.fit(X_train_scaled, y_train)

# 4. Mostrar los mejores parámetros
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)

# 5. Evaluar modelo optimizado
best_rf = grid_search.best_estimator_
best_pred = best_rf.predict(X_test_scaled)

print("\nAccuracy del modelo optimizado:", accuracy_score(y_test, best_pred))
print(classification_report(y_test, best_pred))

# 6. Mejor score de validación cruzada
print("\nMejor accuracy durante la validación cruzada:", grid_search.best_score_)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
